# Skorch pattern for parameter passing 

In [49]:
from skorch import NeuralNetRegressor
from skorch.helper import predefined_split

import argparse
import math
import time
import os

import torch
import torch.nn as nn
import torch.nn.functional as F
from torch.utils.data import TensorDataset
from torch.utils.data import Dataset, DataLoader


import numpy as np

import pdb

### (1) train_step()을 오버라이드하는 방법

In [10]:
# 간단한 네트워크 정의
class SimpleNet(nn.Module):
    def __init__(self, num_features):
        super().__init__()
        self.fc = nn.Linear(num_features, 1)

    def forward(self, x):
        return self.fc(x)

In [11]:
# 사용자 정의 손실 함수 (추가 입력값 활용)
def custom_loss(y_pred, y_true, extra_input):
    # y_pred=[80, 1], y_true=[80, 1], extra_input=[80]
    mse_loss = torch.mean((y_pred - y_true) ** 2)  # 기본 MSE 손실
    penalty = torch.mean(extra_input)  # 예: 특정 입력값의 평균을 패널티로 활용
    return mse_loss + 0.1 * penalty

# skorch `NeuralNetRegressor`를 상속하여 `train_step()`을 오버라이드
class CustomNet(NeuralNetRegressor):
    def train_step(self, batch, **fit_params):
        X, y = batch  # X: 입력 데이터, y: 타겟 값, fit_params={}

        # 모델 예측값
        y_pred = self.forward(X)

        # 추가적인 입력 데이터 활용 가능 (예: X의 첫 번째 feature 사용)
        extra_input = X[:, 0]  # 예: 첫 번째 feature 값을 penalty로 활용

        # 사용자 정의 손실 함수 적용
        loss = custom_loss(y_pred, y, extra_input)

        return {"loss": loss, "y_pred": y_pred}

In [12]:
# skorch 모델 정의
net = CustomNet(
    SimpleNet,
    module__num_features=10,  # 모델에 전달할 인자
    optimizer=torch.optim.Adam,
    max_epochs=5
)

# 데이터 생성
X_train = torch.rand(100, 10)  # 입력값 (100개 샘플, 10개 feature)
y_train = torch.rand(100, 1)   # 타겟값

# 모델 학습
net.fit(X_train, y_train)

  epoch    train_loss    valid_loss     dur
-------  ------------  ------------  ------
      1        0.4673        0.3901  0.0020
      2        0.4673        0.3901  0.0024
      3        0.4673        0.3901  0.0025
      4        0.4673        0.3901  0.0011
      5        0.4673        0.3901  0.0010


<class '__main__.CustomNet'>[initialized](
  module_=SimpleNet(
    (fc): Linear(in_features=10, out_features=1, bias=True)
  ),
)

### (2) fit() 호출 시 추가 데이터 전달 (fit_params 활용)

In [28]:
def custom_loss(y_pred, y_true, extra_input):
    mse_loss = torch.mean((y_pred - y_true) ** 2)  # 기본 MSE 손실
    penalty = torch.mean(extra_input)  # 예: 특정 입력값의 평균을 패널티로 활용
    return mse_loss + 0.1 * penalty
    
class CustomNet(NeuralNetRegressor):
    def train_step(self, batch, **fit_params):
        X, y = batch
        extra_data = fit_params.get("extra_data", torch.zeros_like(y))  # 기본값 설정

        # 모델 예측
        y_pred = self.forward(X)

        # 사용자 정의 손실 함수 적용
        loss = custom_loss(y_pred, y, extra_data)

        return {"loss": loss, "y_pred": y_pred}

# 모델 정의
net = CustomNet(
    SimpleNet,
    module__num_features=10,
    optimizer=torch.optim.Adam,
    max_epochs=5,
    train_split=predefined_split(None),  # ✅ 검증 데이터 사용 안함
)

# 데이터 생성
X_train = torch.rand(100, 10)  # 입력값 (100개 샘플, 10개 feature)
y_train = torch.rand(100, 1)   # 타겟값

# 추가 데이터를 생성
extra_input_values = torch.rand(len(X_train), 1)  # 예제 데이터 생성

# 학습 시 추가 인자 전달
net.fit(X_train, y_train, extra_data=extra_input_values)

  epoch    train_loss     dur
-------  ------------  ------
      1        0.3778  0.0026
      2        0.3778  0.0010
      3        0.3778  0.0010
      4        0.3778  0.0020
      5        0.3778  0.0010


<class '__main__.CustomNet'>[initialized](
  module_=SimpleNet(
    (fc): Linear(in_features=10, out_features=1, bias=True)
  ),
)

### (3) Dataset을 커스터마이즈하여 추가 입력 활용
- validation loss까지 계산 🚀

In [48]:
# 다중 입력을 포함하는 Dataset 정의
class CustomDataset(Dataset):
    def __init__(self, X, y, extra_input):
        self.X = X
        self.y = y
        self.extra_input = extra_input

    def __len__(self):
        return len(self.X)

    def __getitem__(self, idx):
        return self.X[idx], self.y[idx], self.extra_input[idx]  # 추가 입력 반환


# 사용자 정의 손실 함수 (추가 입력값 활용)
def custom_loss(y_pred, y_true, extra_input):
    mse_loss = torch.mean((y_pred - y_true) ** 2)  # 기본 MSE 손실
    penalty = torch.mean(extra_input)  # 예: 특정 입력값의 평균을 패널티로 활용
    return mse_loss + 0.1 * penalty

    
class CustomNet(NeuralNetRegressor):
    def valid_loss(self, X, y, extra_input):
        with torch.no_grad():
            y_pred = self.forward(X)
        # 사용자 정의 손실 함수 적용
        loss = custom_loss(y_pred, y, extra_input)
        return {"loss": loss, "y_pred": y_pred}
        
    def train_step(self, batch, **fit_params):
        X, y, extra_input = batch  # 추가 입력 포함

        # 모델 예측
        y_pred = self.forward(X)

        # 사용자 정의 손실 함수 적용
        loss = custom_loss(y_pred, y, extra_input)

        return {"loss": loss, "y_pred": y_pred}

    def validation_step(self, batch, **fit_params):  # 📌원래는 X, y만 넘어옴 
        X, y, extra_input = batch  # 추가 입력 포함
        return self.valid_loss(X, y, extra_input)


# 데이터 생성
X_train = torch.rand(100, 10)  # 입력값 (100개 샘플, 10개 feature)
y_train = torch.rand(100, 1)   # 타겟값

# 데이터셋 생성
extra_input_values = torch.rand(len(X_train), 1)
train_dataset = CustomDataset(X_train, y_train, extra_input_values)
valid_dataset = CustomDataset(X_train, y_train, extra_input_values)


net = CustomNet(
    SimpleNet,
    module__num_features=10,  # 모델에 전달할 인자
    optimizer=torch.optim.Adam,
    max_epochs=5,
    train_split=predefined_split(valid_dataset),  # ✅ 검증 데이터 사용 
)

# 모델 학습
net.fit(train_dataset, y=None)

  epoch    train_loss    valid_loss     dur
-------  ------------  ------------  ------
      1        0.4673        0.4673  0.0040
      2        0.4673        0.4673  0.0027
      3        0.4673        0.4673  0.0020
      4        0.4673        0.4673  0.0030
      5        0.4673        0.4673  0.0000


<class '__main__.CustomNet'>[initialized](
  module_=SimpleNet(
    (fc): Linear(in_features=10, out_features=1, bias=True)
  ),
)